In [16]:
import sys
sys.path.append('../')
import pytest
import xlwings as xw
import pandas as pd
from pylim import lim
pd.options.display.float_format = "{:,.2f}".format
from xlwings import Range, constants
from oilanalytics.freight import tce_calc

In [2]:
wb = xw.Book(r'C:\dev\oilanalytics\notebooks\Baltic Exchange Tanker TCE Calculator SEP.20.xlsx')
sh = wb.sheets['TCE Calculator']

In [3]:
route = 'tc5'
ds = '2021-10-06'

dd = {
    'FlatRate': 20.5,
    'WorldScale': 103.14,
    'MGO': 355.001,
    'VLSFO': 577.27,
    'HSFO': 225.79,
    'LoadPortCosts': 21181.07,
    'DischargePortCosts': 54347.78,
}

data = pd.DataFrame(dd, index=[pd.to_datetime(ds)])
res = tce_calc.calc(route, data)

res

,FlatRate,WorldScale,MGO,VLSFO,HSFO,LoadPortCosts,DischargePortCosts,Freight_USDMT,GrossFreight,BunkerCost,NetFreight,TotalCosts,NetIncome,TCE
2021-10-06,20.50,103.14,"2,644.76",577.27,225.79,"21,181.07","54,347.78",21.14,"1,177,753.50","853,965.33","1,133,587.74","929,494.18","204,093.56","4,035.93"


In [4]:
routef = route.upper()
sh.range("b7").value = routef
sh.range("C7").value = dd['WorldScale']
sh.range("D7").value = dd['FlatRate']

In [5]:
cmp = sh.range("B79:C86").value
cmp = pd.DataFrame(cmp).set_index(0)
cmp = cmp.rename(columns={1:'Baltic'})
cmp['Code'] = pd.Series()
cmp.loc['Gross Freight', 'Code'] = res['GrossFreight'].iloc[0]
cmp.loc['Bunker Cost', 'Code'] = res['BunkerCost'].iloc[0]
cmp.loc['Net Income', 'Code'] = res['NetIncome'].iloc[0]
cmp.loc['Baltic TCE', 'Code'] = res['TCE'].iloc[0]
cmp

C:\Users\ui906879\AppData\Local\Temp\1/ipykernel_34040/2462558968.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cmp['Code'] = pd.Series()


,Baltic,Code
0,,
Gross Freight,"1,177,753.50","1,177,753.50"
Net Freight,"1,133,587.74",NaN
Bunker Cost,"853,965.28","853,965.33"
Total Expenses,"929,494.12",NaN
Net Income,"204,093.62","204,093.56"
- Gross,"4,909.31",NaN
- Net,"4,035.94",NaN
Baltic TCE,"4,036.00","4,035.93"


In [12]:
# gross freight
assert sh.range("C79").value == pytest.approx(res['GrossFreight'].iloc[0], abs=1e-1)

In [8]:
# bunker cost
assert sh.range("C81").value == pytest.approx(res['BunkerCost'].iloc[0], abs=1e-1)

In [11]:
# tce
assert sh.range("C86").value == pytest.approx(res['TCE'].iloc[0], abs=1e-1)

In [15]:
print("res['GrossFreight'][ds] ==  pytest.approx(%.2f, abs=1e-1)" % res['GrossFreight'].iloc[0])
print("res['BunkerCost'][ds] ==  pytest.approx(%.2f, abs=1e-1))" % res['BunkerCost'].iloc[0])
print("res['TCE'][ds] ==  pytest.approx(%.2f, abs=1e-1))" % res['TCE'].iloc[0])

res['GrossFreight'][ds] ==  pytest.approx(1177753.50, abs=1e-1)
res['BunkerCost'][ds] ==  pytest.approx(853965.33, abs=1e-1))
res['TCE'][ds] ==  pytest.approx(4035.93, abs=1e-1))
